## Blackjack Problem
In this problem we'll be working with a simplified version of [blackjack](https://en.wikipedia.org/wiki/Blackjack) (aka twenty-one). In this version there is one player (who you'll control) and a dealer. Play proceeds as follows:

- The player is dealt two face-up cards. The dealer is dealt one face-up card.
- The player may ask to be dealt another card ('hit') as many times as they wish. If the sum of their cards exceeds 21, they lose the round immediately.
- The dealer then deals additional cards to himself until either:
    - the sum of the dealer's cards exceeds 21, in which case the player wins the round
    - the sum of the dealer's cards is greater than or equal to 17. If the player's total is greater than the dealer's, the player wins. Otherwise, the dealer wins (even in case of a tie).
    
When calculating the sum of cards, Jack, Queen, and King count for 10. Aces can count as 1 or 11 (when referring to a player's 'total' above, we mean the largest total that can be made without exceeding 21. So e.g. A+8 = 19, A+8+8 = 17)

For this problem, you'll write a function representing the player's decision-making strategy in this game. We've provided a very unintelligent implementation below:

In [9]:
import numpy as np
import itertools, random

In [10]:
class Dealer:
    def __init__(self):
        self.name = 'Dealer'
        self.deck = list(itertools.product(range(1,14),['Spade','Heart','Diamond','Club']))
        self.cards, self.nhigh_aces, self.nlow_aces,  self.total_points = [], 0, 0, 0
        self.rank = []
        self.score, self.n1st = 0, 0
    
    def new_game(self, players):
        if type(players) != list:
            list_players = [players, ]
        else:
            list_players = players
        for player in list_players:
            if not callable(getattr(player, 'new_game', None)) or not hasattr(player, 'cards'):
                return 1
            player.new_game()
        
        self._reset_status()
        self.shuffle_deck()
        err = 0
        for player in list_players:
            for i in range(2):
                self.deal(player)
            err = Dealer.count_total_points(player)
        for i in range(2):
            self.deal()
        err += Dealer.count_total_points(self)
        return 1 if err > 0 else 0
        
    
    def _reset_status(self):
        self.deck = list(itertools.product(range(1,14),['Spade','Heart','Diamond','Club']))
        self.cards, self.nhigh_aces, self.nlow_aces,  self.total_points = [], 0, 0, 0
        self.rank = []
    
    def shuffle_deck(self):
        # shuffle the cards
        if not self.cards:
            random.shuffle(self.deck)
    
    def deal(self, to_whom=None):
        # deal one card at a time
        if hasattr(to_whom, 'cards'):
            to_whom.cards.append(self.deck.pop(0))
        else:
            self.cards.append(self.deck.pop(0))
    
    @staticmethod
    def count_total_points(__o):
        if not hasattr(__o, 'nhigh_aces') or not hasattr(__o, 'nlow_aces') or not hasattr(__o, 'total_points'):
            return 1
        point_list = [c[0] if c[0] < 10 else 10 for c in __o.cards]
        other_cards = [p for p in point_list if p > 1]
        naces = len(point_list) - len(other_cards)
        if naces <= 0:
            __o.nhigh_aces = 0
            __o.nlow_aces = 0
            __o.total_points = np.sum(other_cards)
        else:
            __o.nhigh_aces = 1 if np.sum(other_cards) + naces + 10 <= 21 else 0
            __o.nlow_aces = naces - __o.nhigh_aces
            __o.total_points = np.sum(other_cards) + __o.nlow_aces + __o.nhigh_aces * 11
        
        return 0
    
    def should_hit(self):
        # if the dealer should hit the next card, return true
        Dealer.count_total_points(self)
        if self.total_points >= 17:
            return False
        else:
            return True
    
    def gaming(self, players):
        if type(players) != list:
            list_players = [players, ]
        else:
            list_players = players
        for player in list_players:
            if not callable(getattr(player, 'should_hit', None)) or not hasattr(player, 'name') \
                    or not hasattr(player, 'total_points') or not hasattr(player, 'score'):
                return 1
        
        players_on = list_players.copy()
        for player in list_players:
            while player.should_hit(self.total_points):
                self.deal(player)
            if player.total_points > 21:
                self.rank.append(tuple(['Loser', player.name]))
                players_on.remove(player)
        if not players_on:
            self.rank.insert(0, tuple([1, self.name]))
            self.score += len(list_players)
            self.n1st += 1
        else:
            while self.should_hit():
                self.deal()
            if self.total_points > 21:
                self.rank.append(tuple(['Loser', self.name]))
            else:
                players_on.append(self)
            players_on = sorted(players_on, key=lambda x: x.total_points, reverse=True)
            for i in range(len(players_on)):
                if i > 0 and players_on[i].name == self.name:
                    if players_on[i].total_points < players_on[i-1].total_points:
                        break
                    for j in range(i-1, -1, -1):
                        if players_on[i].total_points < players_on[j].total_points:
                            j += 1
                            break
                    popped = players_on.pop(i)
                    players_on.insert(j, popped)
            for i in range(len(players_on)):
                if i > 0 and players_on[i].total_points == players_on[i-1].total_points \
                        and players_on[i-1].name != self.name:
                    self.rank.insert(i, tuple([self.rank[i-1][0], players_on[i].name]))
                    players_on[i].score += len(list_players) - self.rank[i-1][0] + 1
                    if self.rank[i-1][0] <= 1:
                        players_on[i].n1st += 1
                else:
                    self.rank.insert(i, tuple([i+1, players_on[i].name]))
                    players_on[i].score += len(list_players) - i
                    if i <= 0:
                        players_on[i].n1st += 1
        return 0

In [11]:
class Player:
    def __init__(self, name):
        self.name = str(name)
        self.cards, self.nhigh_aces, self.nlow_aces,  self.total_points = [], 0, 0, 0
        self.score, self.n1st = 0, 0
    
    def new_game(self):
        self._reset_status()
    
    def _reset_status(self):
        self.cards, self.nhigh_aces, self.nlow_aces,  self.total_points = [], 0, 0, 0

    def should_hit(self, dealer_tp):
        # if the dealer should hit the next card, return true
        Dealer.count_total_points(self)
        prob_list = [0.0, (4-self.nhigh_aces-self.nlow_aces)/(52-self.nhigh_aces-self.nlow_aces)] \
                    + [1/13] * 8 + [4/13*4] \
                    + [(4-self.nhigh_aces-self.nlow_aces)/(52-self.nhigh_aces-self.nlow_aces)] \
                    + [0.0] * 9
        if self.total_points >= 21:
            return False
        if dealer_tp >= 17 and self.total_points < 18:
            low_points = self.total_points - self.nhigh_aces * 11
            prob_win = np.sum([prob_list[int(x - self.total_points)] for x in range(18, 21)])
            if prob_win > 0.4:
                return True
            else:
                return False
        elif dealer_tp >= 17 and self.total_points >= 18:
            return False
        elif dealer_tp < 17 and self.total_points > 16:
            return False
        else:
            return True
        


In [12]:
player1 = Player('Tom')
players = player1
dealer = Dealer()

In [13]:
for i in range(10000):
    dealer.new_game(players)
    dealer.gaming(players)
    dealer.rank

In [14]:
player1.name, player1.score, player1.n1st

('Tom', 3588, 3588)

In [15]:
# player2.name, player2.score, player2.n1st

In [16]:
dealer.name, dealer.score, dealer.n1st


('Dealer', 6412, 6412)